# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("df", engine)
X = df['message'] 
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26172    The training demonstrated how to enhance micro...
26173    A suitable candidate has been selected and OCH...
26174    Proshika, operating in Cox's Bazar municipalit...
26175    Some 2,000 women protesting against the conduc...
26176    A radical shift in thinking came about as a re...
Name: message, Length: 26177, dtype: object

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [5]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)
model = model_pipeline()

model.fit(X_train, Y_train)



Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred=model.predict(X_test)
labels = np.unique(y_pred)
i=-1
for column in Y_test.columns:
    i=i+1
    print (classification_report(Y_test[column], y_pred[:,i], labels= labels))


              precision    recall  f1-score   support

           0       0.62      0.30      0.41      1557
           1       0.81      0.94      0.87      4941
           2       0.60      0.26      0.36        47

    accuracy                           0.78      6545
   macro avg       0.68      0.50      0.54      6545
weighted avg       0.76      0.78      0.76      6545

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5393
           1       0.77      0.51      0.61      1152
           2       0.00      0.00      0.00         0

   micro avg       0.89      0.89      0.89      6545
   macro avg       0.56      0.49      0.52      6545
weighted avg       0.88      0.89      0.88      6545

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00         0

   micro avg       0

E:\Program_Files\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Program_Files\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5940
           1       0.76      0.51      0.61       605
           2       0.00      0.00      0.00         0

   micro avg       0.94      0.94      0.94      6545
   macro avg       0.57      0.50      0.53      6545
weighted avg       0.93      0.94      0.93      6545

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6451
           1       0.72      0.41      0.53        94
           2       0.00      0.00      0.00         0

   micro avg       0.99      0.99      0.99      6545
   macro avg       0.57      0.47      0.51      6545
weighted avg       0.99      0.99      0.99      6545

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6410
           1       0.55      0.33      0.41       135
           2       0.00      0.00      0.00         0

   micro avg       0

              precision    recall  f1-score   support

           0       0.89      0.96      0.92      5278
           1       0.73      0.50      0.59      1267
           2       0.00      0.00      0.00         0

   micro avg       0.87      0.87      0.87      6545
   macro avg       0.54      0.48      0.50      6545
weighted avg       0.86      0.87      0.86      6545



### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
model.get_params()


{'memory': None,
 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,
                                                                    max_fe

In [14]:
def build_model():

    pipeline = Pipeline([
        ('features',FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    
    parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    } 

    cv = GridSearchCV(pipeline, parameters)
    
    return cv


In [ ]:
print('Building model...')
model = build_model()

print('Training model...')
model.fit(X_train, Y_train)


Building model...
Training model...


In [ ]:
model.best_params_


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print('Evaluating model...')
evaluate_model(model, X_test, Y_test, category_names)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def build_model():

    pipeline = Pipeline([
        ('features',FeatureUnion([
            
            ('message', Pipeline([
                    ('selector', TextSelector(key='message')),
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

            ('genre', Pipeline([
                    ('extract', ColumnExtractor(['genre'])),
                    ('dummy', DummyTransformer())
            ]))
                         ])),
        
        ('clf', MultiOutputClassifier(LinearSVC(multi_class="crammer_singer"), n_jobs=1))
    ])
    
    return pipeline

In [ ]:
model = build_model()

model.fit(X_train, Y_train)

evaluate_model(model, X_test, Y_test, category_names)


### 9. Export your model as a pickle file

In [ ]:
model_filepath = "classifier.p"
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.